<a href="https://colab.research.google.com/github/lightrao/ai-agentic-design-patterns-with-autogen/blob/main/L2/L2_Sequential_Chats_and_Customer_Onboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
%pip install pyautogen chess numpy matplotlib pandas yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.7 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=2440526ddcec124610ed9a240efea93244e7a4100291c2edb357890264cac1b0
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [2]:
import os

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
llm_config={"model": "gpt-3.5-turbo"}

In [4]:
from autogen import ConversableAgent

## Creating the needed agents

In [6]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding_Topic_preference_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [9]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [10]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message":
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [11]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/autogen/agentchat/chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Onboarding_Personal_Information_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: lightrao
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

lightrao

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

I'm sorry, could you please provide me with your name and location to proceed further?

--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to Onboarding_Personal_Information_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: New York
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

New York

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ebce92f9-0fca-4044-9531-9ff046670fff): Maximum turns (2) rea

## Print out the summary

In [12]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'lightrao', 'location': 'New York'}


The customer is interested in reading about AI.


lightrao from New York is interested in reading about AI and may enjoy exploring the concept of Deep Learning, which involves artificial neural networks modelled after the human brain to solve complex problems.




## Print out the cost

In [13]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00017, 'gpt-3.5-turbo-0125': {'cost': 0.00017, 'prompt_tokens': 232, 'completion_tokens': 36, 'total_tokens': 268}}, 'usage_excluding_cached_inference': {'total_cost': 0.00017, 'gpt-3.5-turbo-0125': {'cost': 0.00017, 'prompt_tokens': 232, 'completion_tokens': 36, 'total_tokens': 268}}}


{'usage_including_cached_inference': {'total_cost': 5.4e-05, 'gpt-3.5-turbo-0125': {'cost': 5.4e-05, 'prompt_tokens': 78, 'completion_tokens': 10, 'total_tokens': 88}}, 'usage_excluding_cached_inference': {'total_cost': 5.4e-05, 'gpt-3.5-turbo-0125': {'cost': 5.4e-05, 'prompt_tokens': 78, 'completion_tokens': 10, 'total_tokens': 88}}}


{'usage_including_cached_inference': {'total_cost': 0.000352, 'gpt-3.5-turbo-0125': {'cost': 0.000352, 'prompt_tokens': 290, 'completion_tokens': 138, 'total_tokens': 428}}, 'usage_excluding_cached_inference': {'total_cost': 0.000352, 'gpt-3.5-turbo-0125': {'cost': 0.000352, 'prompt_tokens': 290, 'completion_tokens':